In [2]:
# Cell 0: Install Required Libraries
# ----------------------------------
import sys
import subprocess

def install_package(package):
    try:
        __import__(package)
        print(f"✅ {package} already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} installed successfully")

# Install required packages
install_package('pandas')
install_package('numpy') 
install_package('openpyxl')  # For Excel files

print("\n🚀 All packages ready!")

📦 Installing pandas...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 964.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 15.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 13.4 MB/s eta 0:00:00
✅ pandas installed successfully
✅ numpy already installed
📦 Installing openpyxl...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.6 MB/s eta 0:00:00a 0:00:01
✅ openpyxl installed successfully

🚀 All packages ready!


In [30]:
# Cell 1: Import libraries
import pandas as pd
import numpy as np

print("📚 Libraries imported successfully!")

📚 Libraries imported successfully!


In [31]:
# Cell 2: Read your CSV file
# UPDATE THIS PATH TO YOUR ACTUAL FILE:
file_path = "/Users/darrenang/Downloads/Backtest1.csv"  # ← Change this to your CSV path

# Read the CSV
df = pd.read_csv(file_path)

print(f"✅ File loaded successfully!")
print(f"📊 Shape: {df.shape[0]} rows × {df.shape[1]} columns")

✅ File loaded successfully!
📊 Shape: 43 rows × 59 columns


In [9]:
# Cell 3: Show all column names
print("📋 ALL COLUMNS IN YOUR CSV:")
print("=" * 50)

for i, column in enumerate(df.columns, 1):
    print(f"{i:2d}. {column}")

print(f"\n📊 Total: {len(df.columns)} columns")

📋 ALL COLUMNS IN YOUR CSV:
 1. 1 R X Red Line
 2. Atr Track Back (After Atr)
 3. Atr Track Back (Multiplier)
 4. Account Name
 5. Adjusted Cost
 6. Adjusted Proceeds
 7. Avg Buy Price
 8. Avg Sell Price
 9. Exit Efficiency
10. Best Exit
11. Best Exit Price
12. Best Exit Time
13. Close Date
14. Close Time
15. Commission
16. Duration
17. Entry Candle Size And Color
18. Entry Price
19. Executions
20. Exit Price
21. Gold Candle
22. Gross P&L
23. Trade Risk
24. Initial Target
25. Instrument
26. Spread Type
27. M5, M30, 1 H
28. Macd Condition
29. Max Tp (Atr Multiplier)
30. Mistakes
31. Net P&L
32. Net ROI
33. Open Date
34. Open Time
35. Pips
36. Reward Ratio
37. Playbook
38. Points
39. Position MAE
40. Position MFE
41. Price MAE
42. Price MFE
43. Rsi Position
44. Realized RR
45. Return Per Pip
46. Reviewed
47. Sl Track Back Behind Redline
48. Setups
49. Side
50. Status
51. Symbol
52. Ticks Value
53. Ticks Per Contract
54. Fee
55. Swap
56. Rating
57. Type Of Trades
58. Quantity
59. Zella Sco

In [38]:
# Extract the three key columns
key_columns = [
    'Atr Track Back (Multiplier)',
    'Max Tp (Atr Multiplier)', 
    '1 R X Red Line'  # NEW: Realistic exit data
]

# Create enhanced dataframe
enhanced_df = df[key_columns].copy()

print("🎯 EXTRACTED ENHANCED DATA:")
print("=" * 50)
print(f"📊 Shape: {enhanced_df.shape[0]} rows × {enhanced_df.shape[1]} columns")
print(f"📋 Columns: {list(enhanced_df.columns)}")

🎯 EXTRACTED ENHANCED DATA:
📊 Shape: 43 rows × 3 columns
📋 Columns: ['Atr Track Back (Multiplier)', 'Max Tp (Atr Multiplier)', '1 R X Red Line']


In [39]:
# Analyze what values you have in Red Line column
print("📊 RED LINE VALUES ANALYSIS:")
print("=" * 50)

red_line_values = enhanced_df['1 R X Red Line'].value_counts()
print("Raw values and counts:")
print(red_line_values)

print(f"\n📈 UNIQUE RED LINE VALUES:")
unique_reds = enhanced_df['1 R X Red Line'].unique()
for value in unique_reds:
    if pd.notna(value):
        print(f"   • '{value}'")

📊 RED LINE VALUES ANALYSIS:
Raw values and counts:
1 R X Red Line
lose lose                         17
win win                           13
win lose                           6
win second win if no breakeven     4
Name: count, dtype: int64

📈 UNIQUE RED LINE VALUES:
   • 'win win'
   • 'win lose'
   • 'lose lose'
   • 'win second win if no breakeven'


In [41]:
# Clean all three columns
def clean_atr_value(value):
    """Convert ATR values to numbers"""
    if pd.isna(value):
        return None
    
    value_str = str(value).strip()
    
    if value_str == "-1" or "lost trade" in value_str.lower():
        return -1.0
    
    if value_str.endswith('x'):
        try:
            return float(value_str[:-1])
        except:
            return None
    
    try:
        return float(value_str)
    except:
        return None

def clean_red_line_value(value):
    """Clean Red Line values"""
    if pd.isna(value):
        return None
    
    value_str = str(value).strip().lower()
    
    # Map your categories
    if "win win" in value_str:
        return "win_win"
    elif "win lose" in value_str:
        return "win_lose"  
    elif "win second win if no breakeven" in value_str:
        return "win_win"  # You said treat this as win_win
    elif "lose" in value_str or "lost" in value_str:
        return "lose"
    else:
        return value_str  # Keep original if unrecognized

print("🧹 CLEANING ALL DATA...")

# Clean the data
enhanced_df['Trackback_Clean'] = enhanced_df['Atr Track Back (Multiplier)'].apply(clean_atr_value)
enhanced_df['MaxTP_Clean'] = enhanced_df['Max Tp (Atr Multiplier)'].apply(clean_atr_value)
enhanced_df['RedLine_Clean'] = enhanced_df['1 R X Red Line'].apply(clean_red_line_value)

# Filter out invalid rows
valid_enhanced = enhanced_df[
    enhanced_df['MaxTP_Clean'].notna() & 
    enhanced_df['RedLine_Clean'].notna()
].copy()

print("✅ Data cleaned!")
print(f"📊 Valid trades: {len(valid_enhanced)}")

# Show the breakdown
print(f"\n📊 RED LINE BREAKDOWN:")
red_breakdown = valid_enhanced['RedLine_Clean'].value_counts()
print(red_breakdown)

print(f"\n👀 SAMPLE OF CLEANED DATA:")
print(valid_enhanced[['Atr Track Back (Multiplier)', 'Trackback_Clean', 
                     'Max Tp (Atr Multiplier)', 'MaxTP_Clean',
                     '1 R X Red Line', 'RedLine_Clean']].head(10))

🧹 CLEANING ALL DATA...
✅ Data cleaned!
📊 Valid trades: 40

📊 RED LINE BREAKDOWN:
RedLine_Clean
win_win     17
lose        17
win_lose     6
Name: count, dtype: int64

👀 SAMPLE OF CLEANED DATA:
   Atr Track Back (Multiplier)  Trackback_Clean Max Tp (Atr Multiplier)  \
0                         0.5x              0.5                    1.0x   
1                         0.5x              0.5                    2.1x   
2                         0.5x              0.5                    2.8x   
3                -1 lost trade             -1.0                      -1   
4                -1 lost trade             -1.0                      -1   
6                -1 lost trade             -1.0                      -1   
7                -1 lost trade             -1.0                      -1   
8                -1 lost trade             -1.0                      -1   
9                         0.5x              0.5                    1.6x   
10                        0.5x              0.5          

In [36]:
# Cell 7 (Fixed): Analyze cleaned values - excluding blanks
print("🧹 FILTERING OUT BLANK MAX TP VALUES...")

# Filter out trades where Max TP is blank/null
valid_trades = atr_df[atr_df['MaxTP_Clean'].notna()].copy()

print("📊 CLEANED ATR TRACKBACK VALUES (Valid trades only):")
trackback_clean = valid_trades['Trackback_Clean'].value_counts().sort_index()
print(trackback_clean)

print(f"\n📊 CLEANED MAX TP VALUES (Valid trades only):")
maxtp_clean = valid_trades['MaxTP_Clean'].value_counts().sort_index()
print(maxtp_clean)

# Separate winners and losers (excluding blanks)
winners = valid_trades[valid_trades['Trackback_Clean'] != -1.0]
losers = valid_trades[valid_trades['Trackback_Clean'] == -1.0]

print(f"\n📈 CORRECTED SUMMARY:")
print(f"   • Total VALID trades: {len(valid_trades)} (excluded {len(atr_df) - len(valid_trades)} blanks)")
print(f"   • Winning trades: {len(winners)}")
print(f"   • Losing trades: {len(losers)}")
print(f"   • Win rate: {len(winners)/len(valid_trades)*100:.1f}%")

if len(winners) > 0:
    print(f"   • ATR Trackback range (winners): {winners['Trackback_Clean'].min():.1f}x to {winners['Trackback_Clean'].max():.1f}x")
    print(f"   • Max TP range (winners): {winners['MaxTP_Clean'].min():.1f}x to {winners['MaxTP_Clean'].max():.1f}x")

# Store clean data for optimization
clean_data = valid_trades[['Trackback_Clean', 'MaxTP_Clean']].copy()
print(f"\n✅ Clean dataset ready: {len(clean_data)} trades for ATR optimization")

🧹 FILTERING OUT BLANK MAX TP VALUES...
📊 CLEANED ATR TRACKBACK VALUES (Valid trades only):
Trackback_Clean
-1.0    16
 0.5    12
 0.7     1
 0.8     2
 0.9     1
 1.1     1
 1.3     1
 1.5     1
 1.6     1
 1.7     2
 1.8     1
 2.3     1
Name: count, dtype: int64

📊 CLEANED MAX TP VALUES (Valid trades only):
MaxTP_Clean
-1.0    16
 1.0     1
 1.5     2
 1.6     3
 1.7     1
 2.0     2
 2.1     3
 2.4     2
 2.6     1
 2.7     1
 2.8     1
 2.9     1
 3.4     2
 3.7     1
 3.8     1
 3.9     1
 4.2     1
Name: count, dtype: int64

📈 CORRECTED SUMMARY:
   • Total VALID trades: 40 (excluded 3 blanks)
   • Winning trades: 24
   • Losing trades: 16
   • Win rate: 60.0%
   • ATR Trackback range (winners): 0.5x to 2.3x
   • Max TP range (winners): 1.0x to 4.2x

✅ Clean dataset ready: 40 trades for ATR optimization


In [47]:
# Detailed ATR + R Target Simulation with Full Logging
import pandas as pd
import numpy as np

def simulate_atr_and_r_target_with_logging(data, atr_stop_level, r_target, show_details=False):
    """
    Simulate performance with detailed logging for verification
    """
    results = []
    trade_logs = []
    
    if show_details:
        print(f"\n🔍 DETAILED SIMULATION: {atr_stop_level:.1f}x ATR + {r_target:.1f}R target")
        print("=" * 100)
        print(f"{'Trade':<5} {'Original':<12} {'Trackback':<9} {'MaxTP':<8} {'RedLine':<10} {'Logic':<30} {'Result':<8}")
        print("-" * 100)
    
    for idx, trade in data.iterrows():
        trackback = trade['Trackback_Clean']
        max_tp = trade['MaxTP_Clean']
        red_line = trade['RedLine_Clean']
        
        if pd.isna(trackback) or pd.isna(max_tp) or pd.isna(red_line):
            continue
        
        # Initialize variables for logging
        logic_applied = ""
        original_outcome = f"{red_line}"
        
        # Simulation logic with detailed logging
        if red_line == "lose":
            # Original loser - check if wider stop saves it
            if trackback != -1.0 and trackback < atr_stop_level:
                # Could be saved, but would it reach our R target?
                potential_max = max_tp / atr_stop_level if max_tp > 0 else 0
                if potential_max >= r_target:
                    risk_ratio = r_target  # Hit our R target
                    logic_applied = f"Saved by wide stop, hit {r_target:.1f}R"
                else:
                    risk_ratio = potential_max  # Take whatever available
                    logic_applied = f"Saved but only got {potential_max:.2f}R"
            else:
                risk_ratio = -1.0  # Still a loser
                if trackback == -1.0:
                    logic_applied = "Original loser, still loses"
                else:
                    logic_applied = f"Trackback {trackback:.1f} >= stop {atr_stop_level:.1f}"
                
        else:
            # Original winner (win_win or win_lose)
            if trackback >= atr_stop_level:
                risk_ratio = -1.0  # New stop would stop us out
                logic_applied = f"Stopped out: {trackback:.1f} >= {atr_stop_level:.1f}"
            else:
                # Trade survives, calculate potential with new stop
                potential_max = max_tp / atr_stop_level
                
                if potential_max >= r_target:
                    risk_ratio = r_target  # Take profit at our R target
                    logic_applied = f"Hit R target: {potential_max:.2f} >= {r_target:.1f}"
                else:
                    # Didn't reach our R target
                    if red_line == "win_win":
                        # Originally got full profit, but less than our target
                        risk_ratio = potential_max
                        logic_applied = f"Full profit but < target: {potential_max:.2f}R"
                    elif red_line == "win_lose":
                        # Originally only got 1R, adjust for new stop
                        achievable_r = 1.0 / atr_stop_level
                        risk_ratio = min(achievable_r, potential_max)
                        logic_applied = f"Win-lose: min({achievable_r:.2f}, {potential_max:.2f})"
        
        results.append(risk_ratio)
        
        # Store detailed log
        trade_log = {
            'Trade_Idx': idx,
            'Original_Trackback': trackback,
            'Original_MaxTP': max_tp,
            'Original_RedLine': red_line,
            'ATR_Stop': atr_stop_level,
            'R_Target': r_target,
            'Logic_Applied': logic_applied,
            'Final_RR': risk_ratio
        }
        trade_logs.append(trade_log)
        
        if show_details:
            print(f"{idx:<5} {original_outcome:<12} {trackback:<9} {max_tp:<8} {red_line:<10} {logic_applied:<30} {risk_ratio:+8.2f}")
    
    if show_details:
        winners = [rr for rr in results if rr > 0]
        losers = [rr for rr in results if rr <= 0]
        win_rate = len(winners) / len(results) * 100 if results else 0
        total_profit = sum(winners) if winners else 0
        expectancy = np.mean(results) if results else 0
        
        print(f"\n📊 SUMMARY for {atr_stop_level:.1f}x ATR + {r_target:.1f}R:")
        print(f"   • Total Trades: {len(results)}")
        print(f"   • Winners: {len(winners)} | Losers: {len(losers)}")
        print(f"   • Win Rate: {win_rate:.1f}%")
        print(f"   • Total Profit: {total_profit:.2f}R")
        print(f"   • Expectancy: {expectancy:+.3f}R per trade")
    
    return results, trade_logs

# Test a few specific combinations with detailed logging
import sys
from datetime import datetime

# Create output filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f"atr_simulation_verification_{timestamp}.txt"

# Redirect output to file
original_stdout = sys.stdout
with open(output_filename, 'w', encoding='utf-8') as f:
    sys.stdout = f
    
    print("🔍 ATR SIMULATION DETAILED VERIFICATION LOG")
    print("=" * 80)
    print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Dataset: {len(valid_enhanced)} valid trades")
    print("=" * 80)

    # Test 3 interesting combinations
    test_combinations = [
        (1.0, 2.0),  # 1.0x ATR + 2.0R target
        (1.5, 1.5),  # 1.5x ATR + 1.5R target  
        (2.0, 1.0),  # 2.0x ATR + 1.0R target
    ]

    detailed_results = []

    for atr_stop, r_target in test_combinations:
        results, logs = simulate_atr_and_r_target_with_logging(
            valid_enhanced, atr_stop, r_target, show_details=True
        )
        
        # Calculate metrics
        total_trades = len(results)
        winners = [rr for rr in results if rr > 0]
        losers = [rr for rr in results if rr <= 0]
        
        win_rate = len(winners) / total_trades * 100
        avg_winner = np.mean(winners) if winners else 0
        avg_loser = abs(np.mean(losers)) if losers else 1
        expectancy = np.mean(results)
        total_profit = sum(winners) if winners else 0
        
        detailed_results.append({
            'ATR_Stop': atr_stop,
            'R_Target': r_target,
            'Win_Rate': win_rate,
            'Expectancy': expectancy,
            'Total_Profit': total_profit,
            'Trade_Logs': logs
        })

    # Show ALL trade-by-trade breakdowns for ALL combinations
    print(f"\n📋 COMPLETE TRADE-BY-TRADE VERIFICATION:")
    print("=" * 120)

    for combo_idx, combo in enumerate(detailed_results):
        print(f"\n🎯 COMBINATION {combo_idx + 1}: {combo['ATR_Stop']:.1f}x ATR + {combo['R_Target']:.1f}R")
        print("-" * 120)
        
        verification_df = pd.DataFrame(combo['Trade_Logs'])
        
        print("ALL TRADES - Original Data vs Simulation Results:")
        print(f"{'Trade':<5} {'Orig_TB':<8} {'Orig_MaxTP':<10} {'Orig_RedLine':<12} {'Logic Applied':<40} {'Result':<8}")
        print("-" * 120)

        for _, row in verification_df.iterrows():
            print(f"{row['Trade_Idx']:<5} {row['Original_Trackback']:<8} {row['Original_MaxTP']:<10} {row['Original_RedLine']:<12} {row['Logic_Applied']:<40} {row['Final_RR']:+<8.2f}")

        # Manual verification for this combination
        manual_winners = [log['Final_RR'] for log in combo['Trade_Logs'] if log['Final_RR'] > 0]
        manual_losers = [log['Final_RR'] for log in combo['Trade_Logs'] if log['Final_RR'] <= 0]
        manual_total = sum(manual_winners)
        
        print(f"\n📊 VERIFICATION SUMMARY:")
        print(f"   Manual Winners: {len(manual_winners)} trades")
        print(f"   Manual Losers: {len(manual_losers)} trades") 
        print(f"   Manual Total Profit: {manual_total:.2f}R")
        print(f"   Calculated Total Profit: {combo['Total_Profit']:.2f}R")
        print(f"   Difference: {abs(manual_total - combo['Total_Profit']):.4f}R")
        print(f"   Match: {'✅ CORRECT' if abs(manual_total - combo['Total_Profit']) < 0.01 else '❌ ERROR'}")
        
        print(f"\n   Manual Win Rate: {len(manual_winners)/(len(manual_winners)+len(manual_losers))*100:.1f}%")
        print(f"   Calculated Win Rate: {combo['Win_Rate']:.1f}%")
        print(f"   Manual Expectancy: {(manual_total + sum(manual_losers))/(len(manual_winners)+len(manual_losers)):.3f}")
        print(f"   Calculated Expectancy: {combo['Expectancy']:.3f}")

    # Overall verification checklist
    print(f"\n❓ VERIFICATION CHECKLIST:")
    print("=" * 50)
    print("1. Do the 'Logic Applied' descriptions make sense?")
    print("2. Are lose->lose conversions correct?")
    print("3. Are win_win profits calculated properly?") 
    print("4. Are win_lose adjustments accurate?")
    print("5. Do trackback comparisons work correctly?")
    print("6. Do manual calculations match automated results?")
    
    print(f"\n📁 ORIGINAL DATA SAMPLE FOR REFERENCE:")
    print("=" * 80)
    print("First 10 trades from your dataset:")
    sample_data = valid_enhanced[['Atr Track Back (Multiplier)', 'Trackback_Clean', 
                                  'Max Tp (Atr Multiplier)', 'MaxTP_Clean',
                                  '1 R X Red Line', 'RedLine_Clean']].head(10)
    print(sample_data.to_string())

# Restore stdout
sys.stdout = original_stdout

print(f"✅ VERIFICATION LOG SAVED!")
print(f"📁 File: {output_filename}")
print(f"📊 Contains detailed breakdown of all trades for 3 combinations")
print(f"🔍 Review this file to verify simulation accuracy")

# Also create a summary for quick reference
summary_filename = f"atr_simulation_summary_{timestamp}.txt"
with open(summary_filename, 'w', encoding='utf-8') as f:
    f.write("🎯 ATR SIMULATION SUMMARY\n")
    f.write("=" * 40 + "\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    
    for i, combo in enumerate(detailed_results):
        f.write(f"Combination {i+1}: {combo['ATR_Stop']:.1f}x ATR + {combo['R_Target']:.1f}R\n")
        f.write(f"  Win Rate: {combo['Win_Rate']:.1f}%\n")
        f.write(f"  Expectancy: {combo['Expectancy']:+.3f}R\n")
        f.write(f"  Total Profit: {combo['Total_Profit']:.2f}R\n\n")

print(f"✅ SUMMARY SAVED!")
print(f"📁 File: {summary_filename}")
print(f"\n🚀 Review both files to verify the simulation is working correctly!")

✅ VERIFICATION LOG SAVED!
📁 File: atr_simulation_verification_20250607_222314.txt
📊 Contains detailed breakdown of all trades for 3 combinations
🔍 Review this file to verify simulation accuracy
✅ SUMMARY SAVED!
📁 File: atr_simulation_summary_20250607_222314.txt

🚀 Review both files to verify the simulation is working correctly!


In [50]:
# Complete ATR + R Target Optimization with FULL Logging for ALL Combinations
import pandas as pd
import numpy as np
import sys
from datetime import datetime

def simulate_atr_and_r_target_with_logging(data, atr_stop_level, r_target, log_file=None):
    """
    Simulate performance with detailed logging for verification
    """
    results = []
    trade_logs = []
    
    if log_file:
        log_file.write(f"\n🔍 COMBINATION: {atr_stop_level:.1f}x ATR + {r_target:.1f}R target\n")
        log_file.write("=" * 100 + "\n")
        log_file.write(f"{'Trade':<5} {'Orig_TB':<8} {'Orig_MaxTP':<10} {'Orig_RedLine':<12} {'Logic Applied':<45} {'Result':<8}\n")
        log_file.write("-" * 100 + "\n")
    
    for idx, trade in data.iterrows():
        trackback = trade['Trackback_Clean']
        max_tp = trade['MaxTP_Clean']
        red_line = trade['RedLine_Clean']
        
        if pd.isna(trackback) or pd.isna(max_tp) or pd.isna(red_line):
            continue
        
        # Initialize variables for logging
        logic_applied = ""
        
        # Simulation logic with detailed logging
        if red_line == "lose":
            # Original loser - check if wider stop saves it
            if trackback != -1.0 and trackback < atr_stop_level:
                # Could be saved, but would it reach our R target?
                potential_max = max_tp / atr_stop_level if max_tp > 0 else 0
                if potential_max >= r_target:
                    risk_ratio = r_target  # Hit our R target
                    logic_applied = f"Saved by wide stop, hit {r_target:.1f}R target"
                else:
                    risk_ratio = -1.0  # Saved but didn't reach target
                    logic_applied = f"Saved but missed target: {potential_max:.2f} < {r_target:.1f}R"
            else:
                risk_ratio = -1.0  # Still a loser
                if trackback == -1.0:
                    logic_applied = "Original loser (-1), still loses"
                else:
                    logic_applied = f"Trackback {trackback:.1f} >= stop {atr_stop_level:.1f}, stopped"
                
        else:
            # Original winner (win_win or win_lose)
            if trackback >= atr_stop_level:
                risk_ratio = -1.0  # New stop would stop us out
                logic_applied = f"Stopped out: trackback {trackback:.1f} >= stop {atr_stop_level:.1f}"
            else:
                # Trade survives, calculate potential with new stop
                potential_max = max_tp / atr_stop_level
                
                if potential_max >= r_target:
                    risk_ratio = r_target  # Hit our R target
                    logic_applied = f"Hit R target: {potential_max:.2f} >= {r_target:.1f}, take {r_target:.1f}R"
                else:
                    # Didn't reach our R target - NO PROFIT TAKEN
                    risk_ratio = -1.0  # Either stopped out or didn't reach target
                    logic_applied = f"Missed R target: {potential_max:.2f} < {r_target:.1f}R, no profit"
        
        results.append(risk_ratio)
        
        # Store detailed log
        trade_log = {
            'Trade_Idx': idx,
            'ATR_Stop': atr_stop_level,
            'R_Target': r_target,
            'Original_Trackback': trackback,
            'Original_MaxTP': max_tp,
            'Original_RedLine': red_line,
            'Logic_Applied': logic_applied,
            'Final_RR': risk_ratio
        }
        trade_logs.append(trade_log)
        
        if log_file:
            log_file.write(f"{idx:<5} {trackback:<8} {max_tp:<10} {red_line:<12} {logic_applied:<45} {risk_ratio:+8.2f}\n")
    
    # Calculate summary metrics
    total_trades = len(results)
    winners = [rr for rr in results if rr > 0]
    losers = [rr for rr in results if rr <= 0]
    
    win_rate = len(winners) / total_trades * 100 if total_trades > 0 else 0
    avg_winner = np.mean(winners) if winners else 0
    avg_loser = abs(np.mean(losers)) if losers else 1
    expectancy = np.mean(results) if results else 0
    total_profit = sum(winners) if winners else 0
    profit_factor = total_profit / abs(sum(losers)) if losers else 0
    
    if log_file:
        log_file.write(f"\n📊 SUMMARY:\n")
        log_file.write(f"   • Total Trades: {total_trades}\n")
        log_file.write(f"   • Winners: {len(winners)} | Losers: {len(losers)}\n")
        log_file.write(f"   • Win Rate: {win_rate:.1f}%\n")
        log_file.write(f"   • Avg Winner: {avg_winner:.3f}R\n")
        log_file.write(f"   • Avg Loser: {avg_loser:.3f}R\n")
        log_file.write(f"   • Total Profit: {total_profit:.2f}R\n")
        log_file.write(f"   • Expectancy: {expectancy:+.3f}R per trade\n")
        log_file.write(f"   • Profit Factor: {profit_factor:.2f}\n")
        log_file.write("\n" + "="*100 + "\n")
    
    return {
        'ATR_Stop': atr_stop_level,
        'R_Target': r_target,
        'Win_Rate': win_rate,
        'Avg_Winner': avg_winner,
        'Avg_Loser': avg_loser,
        'Expectancy': expectancy,
        'Profit_Factor': profit_factor,
        'Total_Profit': total_profit,
        'Winners': len(winners),
        'Losers': len(losers),
        'Trade_Logs': trade_logs
    }

# Complete optimization ranges
atr_levels = np.arange(0.6, 3.1, 0.1).round(1).tolist()  # 0.6 to 3.0 in 0.1 increments
r_targets = np.arange(1.0, 4.1, 0.1).round(1).tolist()   # 1.0 to 4.0 in 0.1 increments

total_combinations = len(atr_levels) * len(r_targets)

print(f"🚀 RUNNING COMPLETE OPTIMIZATION WITH FULL LOGGING")
print(f"📊 Testing {len(atr_levels)} ATR levels × {len(r_targets)} R targets = {total_combinations} combinations")
print(f"📁 This will generate detailed logs for ALL combinations...")

# Create timestamped filenames
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
detailed_log_filename = f"complete_atr_optimization_detailed_{timestamp}.txt"
summary_filename = f"complete_atr_optimization_summary_{timestamp}.txt"
results_csv_filename = f"complete_atr_optimization_results_{timestamp}.csv"

print(f"📝 Detailed log: {detailed_log_filename}")
print(f"📊 Summary: {summary_filename}")
print(f"📈 Results CSV: {results_csv_filename}")

# Run complete optimization with logging
optimization_results = []
combination_count = 0

# Open detailed log file
with open(detailed_log_filename, 'w', encoding='utf-8') as detailed_log:
    detailed_log.write("🔍 COMPLETE ATR + R TARGET OPTIMIZATION - DETAILED LOG\n")
    detailed_log.write("=" * 80 + "\n")
    detailed_log.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    detailed_log.write(f"Dataset: {len(valid_enhanced)} valid trades\n")
    detailed_log.write(f"ATR Range: {atr_levels[0]:.1f} to {atr_levels[-1]:.1f} (increments: 0.1)\n")
    detailed_log.write(f"R Target Range: {r_targets[0]:.1f} to {r_targets[-1]:.1f} (increments: 0.1)\n")
    detailed_log.write(f"Total Combinations: {total_combinations}\n")
    detailed_log.write("=" * 80 + "\n\n")
    
    # Run all combinations
    for atr_level in atr_levels:
        for r_target in r_targets:
            combination_count += 1
            
            # Show progress
            if combination_count % 50 == 0:
                print(f"📈 Progress: {combination_count}/{total_combinations} combinations completed ({combination_count/total_combinations*100:.1f}%)")
            
            # Run simulation with logging
            result = simulate_atr_and_r_target_with_logging(
                valid_enhanced, atr_level, r_target, detailed_log
            )
            
            optimization_results.append(result)

print(f"✅ Completed all {total_combinations} combinations!")

# Convert to DataFrame for analysis
opt_df = pd.DataFrame([{k: v for k, v in result.items() if k != 'Trade_Logs'} 
                       for result in optimization_results])

# Save results to CSV
opt_df.to_csv(results_csv_filename, index=False)

# Find top results
top_20_expectancy = opt_df.nlargest(20, 'Expectancy')
top_20_profit = opt_df.nlargest(20, 'Total_Profit')
top_20_winrate = opt_df.nlargest(20, 'Win_Rate')

# Create summary file
with open(summary_filename, 'w', encoding='utf-8') as summary:
    summary.write("🎯 COMPLETE ATR + R TARGET OPTIMIZATION - SUMMARY\n")
    summary.write("=" * 60 + "\n")
    summary.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    summary.write(f"Total combinations tested: {total_combinations}\n")
    summary.write(f"Dataset: {len(valid_enhanced)} valid trades\n\n")
    
    # Best overall results
    best_expectancy = opt_df.loc[opt_df['Expectancy'].idxmax()]
    best_profit = opt_df.loc[opt_df['Total_Profit'].idxmax()]
    best_winrate = opt_df.loc[opt_df['Win_Rate'].idxmax()]
    
    summary.write("🏆 OPTIMAL RESULTS:\n")
    summary.write("-" * 40 + "\n")
    summary.write(f"🎯 Best Expectancy: {best_expectancy['ATR_Stop']:.1f}x ATR + {best_expectancy['R_Target']:.1f}R\n")
    summary.write(f"   Expectancy: {best_expectancy['Expectancy']:+.3f}R per trade\n")
    summary.write(f"   Win Rate: {best_expectancy['Win_Rate']:.1f}%\n")
    summary.write(f"   Total Profit: {best_expectancy['Total_Profit']:.2f}R\n\n")
    
    summary.write(f"💰 Highest Total Profit: {best_profit['ATR_Stop']:.1f}x ATR + {best_profit['R_Target']:.1f}R\n")
    summary.write(f"   Total Profit: {best_profit['Total_Profit']:.2f}R\n")
    summary.write(f"   Expectancy: {best_profit['Expectancy']:+.3f}R\n")
    summary.write(f"   Win Rate: {best_profit['Win_Rate']:.1f}%\n\n")
    
    summary.write(f"🎪 Highest Win Rate: {best_winrate['ATR_Stop']:.1f}x ATR + {best_winrate['R_Target']:.1f}R\n")
    summary.write(f"   Win Rate: {best_winrate['Win_Rate']:.1f}%\n")
    summary.write(f"   Expectancy: {best_winrate['Expectancy']:+.3f}R\n")
    summary.write(f"   Total Profit: {best_winrate['Total_Profit']:.2f}R\n\n")
    
    # Top 20 by expectancy
    summary.write("📊 TOP 20 COMBINATIONS BY EXPECTANCY:\n")
    summary.write("-" * 60 + "\n")
    summary.write(f"{'Rank':<4} {'ATR':<6} {'R Target':<8} {'Expectancy':<10} {'Win Rate':<8} {'Total Profit':<12}\n")
    summary.write("-" * 60 + "\n")
    
    for i, (_, row) in enumerate(top_20_expectancy.iterrows(), 1):
        summary.write(f"{i:<4} {row['ATR_Stop']:<6.1f} {row['R_Target']:<8.1f} {row['Expectancy']:<10.3f} {row['Win_Rate']:<8.1f}% {row['Total_Profit']:<12.2f}\n")

print(f"\n🎊 COMPLETE OPTIMIZATION FINISHED!")
print("=" * 50)
print(f"📁 Files created:")
print(f"   • {detailed_log_filename} - Complete trade-by-trade logs")
print(f"   • {summary_filename} - Key results and top combinations")
print(f"   • {results_csv_filename} - All results in spreadsheet format")

print(f"\n🏆 BEST RESULTS:")
print(f"🎯 Optimal: {best_expectancy['ATR_Stop']:.1f}x ATR + {best_expectancy['R_Target']:.1f}R (Expectancy: {best_expectancy['Expectancy']:+.3f})")
print(f"💰 Max Profit: {best_profit['ATR_Stop']:.1f}x ATR + {best_profit['R_Target']:.1f}R (Profit: {best_profit['Total_Profit']:.2f}R)")

print(f"\n✨ Review the detailed log to verify all calculations are correct!")

🚀 RUNNING COMPLETE OPTIMIZATION WITH FULL LOGGING
📊 Testing 25 ATR levels × 31 R targets = 775 combinations
📁 This will generate detailed logs for ALL combinations...
📝 Detailed log: complete_atr_optimization_detailed_20250607_223402.txt
📊 Summary: complete_atr_optimization_summary_20250607_223402.txt
📈 Results CSV: complete_atr_optimization_results_20250607_223402.csv
📈 Progress: 50/775 combinations completed (6.5%)
📈 Progress: 100/775 combinations completed (12.9%)
📈 Progress: 150/775 combinations completed (19.4%)
📈 Progress: 200/775 combinations completed (25.8%)
📈 Progress: 250/775 combinations completed (32.3%)
📈 Progress: 300/775 combinations completed (38.7%)
📈 Progress: 350/775 combinations completed (45.2%)
📈 Progress: 400/775 combinations completed (51.6%)
📈 Progress: 450/775 combinations completed (58.1%)
📈 Progress: 500/775 combinations completed (64.5%)
📈 Progress: 550/775 combinations completed (71.0%)
📈 Progress: 600/775 combinations completed (77.4%)
📈 Progress: 650/77

In [ ]:
# Complete ATR + R Target Optimization with FULL Logging for ALL Combinations
import pandas as pd
import numpy as np
import sys
from datetime import datetime

def simulate_atr_and_r_target_with_logging(data, atr_stop_level, r_target, log_file=None):
    """
    Simulate performance with detailed logging for verification
    r_target can be a number (e.g., 2.0) or "redline" for original exit strategy
    """
    results = []
    trade_logs = []
    
    target_display = "Redline" if r_target == "redline" else f"{r_target:.1f}R"
    
    if log_file:
        log_file.write(f"\n🔍 COMBINATION: {atr_stop_level:.1f}x ATR + {target_display} target\n")
        log_file.write("=" * 100 + "\n")
        log_file.write(f"{'Trade':<5} {'Orig_TB':<8} {'Orig_MaxTP':<10} {'Orig_RedLine':<12} {'Logic Applied':<45} {'Result':<8}\n")
        log_file.write("-" * 100 + "\n")
    
    for idx, trade in data.iterrows():
        trackback = trade['Trackback_Clean']
        max_tp = trade['MaxTP_Clean']
        red_line = trade['RedLine_Clean']
        
        if pd.isna(trackback) or pd.isna(max_tp) or pd.isna(red_line):
            continue
        
        # Initialize variables for logging
        logic_applied = ""
        
        # Simulation logic with detailed logging
        if red_line == "lose":
            # Original loser - check if wider stop saves it
            if trackback != -1.0 and trackback < atr_stop_level:
                # Could be saved, but would it reach our target?
                potential_max = max_tp / atr_stop_level if max_tp > 0 else 0
                
                if r_target == "redline":
                    # For redline, if saved, assume they would have gotten at least 1R
                    risk_ratio = 1.0 / atr_stop_level  # Conservative: assume 1R equivalent
                    logic_applied = f"Saved by wide stop, assume 1R equiv = {risk_ratio:.2f}R"
                else:
                    # Fixed R target
                    if potential_max >= r_target:
                        risk_ratio = r_target  # Hit our R target
                        logic_applied = f"Saved by wide stop, hit {r_target:.1f}R target"
                    else:
                        risk_ratio = -1.0  # Saved but didn't reach target
                        logic_applied = f"Saved but missed target: {potential_max:.2f} < {r_target:.1f}R"
            else:
                risk_ratio = -1.0  # Still a loser
                if trackback == -1.0:
                    logic_applied = "Original loser (-1), still loses"
                else:
                    logic_applied = f"Trackback {trackback:.1f} >= stop {atr_stop_level:.1f}, stopped"
                
        else:
            # Original winner (win_win or win_lose)
            if trackback >= atr_stop_level:
                risk_ratio = -1.0  # New stop would stop us out
                logic_applied = f"Stopped out: trackback {trackback:.1f} >= stop {atr_stop_level:.1f}"
            else:
                # Trade survives, calculate result based on target type
                if r_target == "redline":
                    # Use original redline strategy results, scaled for new stop
                    if red_line == "win_win":
                        # Originally got full profit to redline
                        risk_ratio = max_tp / atr_stop_level
                        logic_applied = f"Redline win_win: {max_tp:.1f}x/{atr_stop_level:.1f} = {risk_ratio:.2f}R"
                    elif red_line == "win_lose":
                        # Originally only got 1R (missed redline)
                        risk_ratio = 1.0 / atr_stop_level
                        logic_applied = f"Redline win_lose: 1R/{atr_stop_level:.1f} = {risk_ratio:.2f}R"
                else:
                    # Fixed R target strategy
                    potential_max = max_tp / atr_stop_level
                    
                    if potential_max >= r_target:
                        risk_ratio = r_target  # Hit our R target
                        logic_applied = f"Hit R target: {potential_max:.2f} >= {r_target:.1f}, take {r_target:.1f}R"
                    else:
                        # Didn't reach our R target - NO PROFIT TAKEN
                        risk_ratio = -1.0  # Either stopped out or didn't reach target
                        logic_applied = f"Missed R target: {potential_max:.2f} < {r_target:.1f}R, no profit"
        
        results.append(risk_ratio)
        
        # Store detailed log
        trade_log = {
            'Trade_Idx': idx,
            'ATR_Stop': atr_stop_level,
            'R_Target': target_display,
            'Original_Trackback': trackback,
            'Original_MaxTP': max_tp,
            'Original_RedLine': red_line,
            'Logic_Applied': logic_applied,
            'Final_RR': risk_ratio
        }
        trade_logs.append(trade_log)
        
        if log_file:
            log_file.write(f"{idx:<5} {trackback:<8} {max_tp:<10} {red_line:<12} {logic_applied:<45} {risk_ratio:+8.2f}\n")
    
    # Calculate summary metrics
    total_trades = len(results)
    winners = [rr for rr in results if rr > 0]
    losers = [rr for rr in results if rr <= 0]
    
    win_rate = len(winners) / total_trades * 100 if total_trades > 0 else 0
    avg_winner = np.mean(winners) if winners else 0
    avg_loser = abs(np.mean(losers)) if losers else 1
    expectancy = np.mean(results) if results else 0
    total_profit = sum(winners) if winners else 0
    profit_factor = total_profit / abs(sum(losers)) if losers else 0
    
    if log_file:
        log_file.write(f"\n📊 SUMMARY:\n")
        log_file.write(f"   • Total Trades: {total_trades}\n")
        log_file.write(f"   • Winners: {len(winners)} | Losers: {len(losers)}\n")
        log_file.write(f"   • Win Rate: {win_rate:.1f}%\n")
        log_file.write(f"   • Avg Winner: {avg_winner:.3f}R\n")
        log_file.write(f"   • Avg Loser: {avg_loser:.3f}R\n")
        log_file.write(f"   • Total Profit: {total_profit:.2f}R\n")
        log_file.write(f"   • Expectancy: {expectancy:+.3f}R per trade\n")
        log_file.write(f"   • Profit Factor: {profit_factor:.2f}\n")
        log_file.write("\n" + "="*100 + "\n")
    
    return {
        'ATR_Stop': atr_stop_level,
        'R_Target': r_target if isinstance(r_target, str) else r_target,
        'Win_Rate': win_rate,
        'Avg_Winner': avg_winner,
        'Avg_Loser': avg_loser,
        'Expectancy': expectancy,
        'Profit_Factor': profit_factor,
        'Total_Profit': total_profit,
        'Winners': len(winners),
        'Losers': len(losers),
        'Trade_Logs': trade_logs
    }

# Complete optimization ranges
atr_levels = np.arange(0.6, 3.1, 0.1).round(1).tolist()  # 0.6 to 3.0 in 0.1 increments
r_targets = ["redline"] + np.arange(1.0, 4.1, 0.1).round(1).tolist()   # Redline + 1.0 to 4.0 in 0.1 increments

total_combinations = len(atr_levels) * len(r_targets)

print(f"🚀 RUNNING COMPLETE OPTIMIZATION WITH FULL LOGGING")
print(f"📊 Testing {len(atr_levels)} ATR levels × {len(r_targets)} targets = {total_combinations} combinations")
print(f"🎯 Targets: Redline strategy + Fixed R targets (1.0R to 4.0R)")
print(f"📁 This will generate detailed logs for ALL combinations...")

# Create timestamped filenames
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
detailed_log_filename = f"complete_atr_optimization_detailed_{timestamp}.txt"
summary_filename = f"complete_atr_optimization_summary_{timestamp}.txt"
results_csv_filename = f"complete_atr_optimization_results_{timestamp}.csv"

print(f"📝 Detailed log: {detailed_log_filename}")
print(f"📊 Summary: {summary_filename}")
print(f"📈 Results CSV: {results_csv_filename}")

# Run complete optimization with logging
optimization_results = []
combination_count = 0

# Open detailed log file
with open(detailed_log_filename, 'w', encoding='utf-8') as detailed_log:
    detailed_log.write("🔍 COMPLETE ATR + R TARGET OPTIMIZATION - DETAILED LOG\n")
    detailed_log.write("=" * 80 + "\n")
    detailed_log.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    detailed_log.write(f"Dataset: {len(valid_enhanced)} valid trades\n")
    detailed_log.write(f"ATR Range: {atr_levels[0]:.1f} to {atr_levels[-1]:.1f} (increments: 0.1)\n")
    detailed_log.write(f"R Target Range: Redline + {r_targets[1]:.1f} to {r_targets[-1]:.1f} (increments: 0.1)\n")
    detailed_log.write(f"Total Combinations: {total_combinations}\n")
    detailed_log.write("=" * 80 + "\n")
    detailed_log.write("🎯 REDLINE STRATEGY: Uses your original exit logic (win_win = full TP, win_lose = 1R only)\n")
    detailed_log.write("🎯 FIXED R TARGETS: Binary outcome (hit target = +R, miss target = -1R)\n")
    detailed_log.write("=" * 80 + "\n\n")
    
    # Run all combinations
    for atr_level in atr_levels:
        for r_target in r_targets:
            combination_count += 1
            
            # Show progress
            if combination_count % 50 == 0:
                print(f"📈 Progress: {combination_count}/{total_combinations} combinations completed ({combination_count/total_combinations*100:.1f}%)")
            
            # Run simulation with logging
            result = simulate_atr_and_r_target_with_logging(
                valid_enhanced, atr_level, r_target, detailed_log
            )
            
            # Ensure R_Target is properly stored
            result['R_Target'] = r_target
            
            optimization_results.append(result)

print(f"✅ Completed all {total_combinations} combinations!")

# Convert to DataFrame for analysis
opt_df = pd.DataFrame([{k: v for k, v in result.items() if k != 'Trade_Logs'} 
                       for result in optimization_results])

# Save results to CSV
opt_df.to_csv(results_csv_filename, index=False)

# Find top results
top_20_expectancy = opt_df.nlargest(20, 'Expectancy')
top_20_profit = opt_df.nlargest(20, 'Total_Profit')
top_20_winrate = opt_df.nlargest(20, 'Win_Rate')

# Create summary file
with open(summary_filename, 'w', encoding='utf-8') as summary:
    summary.write("🎯 COMPLETE ATR + R TARGET OPTIMIZATION - SUMMARY\n")
    summary.write("=" * 60 + "\n")
    summary.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    summary.write(f"Total combinations tested: {total_combinations}\n")
    summary.write(f"Dataset: {len(valid_enhanced)} valid trades\n")
    summary.write(f"Targets tested: Redline + Fixed R targets (1.0R to 4.0R)\n\n")
    
    summary.write("🎯 TARGET EXPLANATIONS:\n")
    summary.write("   • Redline: Uses original exit strategy (win_win=full TP, win_lose=1R)\n")
    summary.write("   • Fixed R: Binary outcome (hit target or -1R loss)\n\n")
    
    # Best overall results
    best_expectancy = opt_df.loc[opt_df['Expectancy'].idxmax()]
    best_profit = opt_df.loc[opt_df['Total_Profit'].idxmax()]
    best_winrate = opt_df.loc[opt_df['Win_Rate'].idxmax()]
    
    summary.write("🏆 OPTIMAL RESULTS:\n")
    summary.write("-" * 40 + "\n")
    summary.write(f"🎯 Best Expectancy: {best_expectancy['ATR_Stop']:.1f}x ATR + {best_expectancy['R_Target']} target\n")
    summary.write(f"   Expectancy: {best_expectancy['Expectancy']:+.3f}R per trade\n")
    summary.write(f"   Win Rate: {best_expectancy['Win_Rate']:.1f}%\n")
    summary.write(f"   Total Profit: {best_expectancy['Total_Profit']:.2f}R\n\n")
    
    summary.write(f"💰 Highest Total Profit: {best_profit['ATR_Stop']:.1f}x ATR + {best_profit['R_Target']} target\n")
    summary.write(f"   Total Profit: {best_profit['Total_Profit']:.2f}R\n")
    summary.write(f"   Expectancy: {best_profit['Expectancy']:+.3f}R\n")
    summary.write(f"   Win Rate: {best_profit['Win_Rate']:.1f}%\n\n")
    
    summary.write(f"🎪 Highest Win Rate: {best_winrate['ATR_Stop']:.1f}x ATR + {best_winrate['R_Target']} target\n")
    summary.write(f"   Win Rate: {best_winrate['Win_Rate']:.1f}%\n")
    summary.write(f"   Expectancy: {best_winrate['Expectancy']:+.3f}R\n")
    summary.write(f"   Total Profit: {best_winrate['Total_Profit']:.2f}R\n\n")
    
    # Top 20 by expectancy
    summary.write("📊 TOP 20 COMBINATIONS BY EXPECTANCY:\n")
    summary.write("-" * 60 + "\n")
    summary.write(f"{'Rank':<4} {'ATR':<6} {'R Target':<8} {'Expectancy':<10} {'Win Rate':<8} {'Total Profit':<12}\n")
    summary.write("-" * 60 + "\n")
    
    for i, (_, row) in enumerate(top_20_expectancy.iterrows(), 1):
        target_display = str(row['R_Target']) if row['R_Target'] == 'redline' else f"{row['R_Target']:.1f}"
        summary.write(f"{i:<4} {row['ATR_Stop']:<6.1f} {target_display:<8} {row['Expectancy']:<10.3f} {row['Win_Rate']:<8.1f}% {row['Total_Profit']:<12.2f}\n")

print(f"\n🎊 COMPLETE OPTIMIZATION FINISHED!")
print("=" * 50)
print(f"📁 Files created:")
print(f"   • {detailed_log_filename} - Complete trade-by-trade logs")
print(f"   • {summary_filename} - Key results and top combinations")
print(f"   • {results_csv_filename} - All results in spreadsheet format")

print(f"\n🏆 BEST RESULTS:")
print(f"🎯 Optimal: {best_expectancy['ATR_Stop']:.1f}x ATR + {best_expectancy['R_Target']} target (Expectancy: {best_expectancy['Expectancy']:+.3f})")
print(f"💰 Max Profit: {best_profit['ATR_Stop']:.1f}x ATR + {best_profit['R_Target']} target (Profit: {best_profit['Total_Profit']:.2f}R)")

print(f"\n✨ Review the detailed log to verify all calculations are correct!")

🚀 RUNNING COMPLETE OPTIMIZATION WITH FULL LOGGING
📊 Testing 25 ATR levels × 32 targets = 800 combinations
🎯 Targets: Enhanced Redline strategies + Fixed R targets
   • Redline strategies: Different R amounts for win_lose trades
   • Fixed R targets: Binary outcome (hit target or -1R)
📁 This will generate detailed logs for ALL combinations...
📝 Detailed log: complete_atr_optimization_detailed_20250607_232714.txt
📊 Summary: complete_atr_optimization_summary_20250607_232714.txt
📈 Results CSV: complete_atr_optimization_results_20250607_232714.csv
📈 Progress: 50/800 combinations completed (6.2%)
📈 Progress: 100/800 combinations completed (12.5%)
📈 Progress: 150/800 combinations completed (18.8%)
📈 Progress: 200/800 combinations completed (25.0%)
📈 Progress: 250/800 combinations completed (31.2%)
📈 Progress: 300/800 combinations completed (37.5%)
📈 Progress: 350/800 combinations completed (43.8%)
📈 Progress: 400/800 combinations completed (50.0%)
📈 Progress: 450/800 combinations completed (5

In [ ]:
# Complete ATR + R Target Optimization with FULL Logging for ALL Combinations
import pandas as pd
import numpy as np
import sys
from datetime import datetime

def simulate_atr_and_r_target_with_logging(data, atr_stop_level, r_target, log_file=None):
    """
    Simulate performance with detailed logging for verification
    r_target can be:
    - A number (e.g., 2.0) for fixed R target strategy
    - "redline_clean_X.X" (e.g., "redline_clean_2.4") for redline strategy with X.X R target (in current ATR terms)
    """
    results = []
    trade_logs = []
    
    # Parse target type
    if isinstance(r_target, str) and r_target.startswith("redline_clean_"):
        is_redline = True
        target_r_current_atr = float(r_target.split("_")[2])  # This is already in current ATR terms
        # Convert to original 1.0x ATR terms for comparison with MaxTP
        original_win_lose_r = target_r_current_atr * atr_stop_level
        target_display = f"Redline({target_r_current_atr:.1f}R)"
    else:
        is_redline = False
        original_win_lose_r = None
        target_r_current_atr = r_target  # Already in current ATR terms
        target_display = f"{r_target:.1f}R"
    
    if log_file:
        log_file.write(f"\n🔍 COMBINATION: {atr_stop_level:.1f}x ATR + {target_display} target\n")
        log_file.write("=" * 100 + "\n")
        log_file.write(f"{'Trade':<5} {'Orig_TB':<8} {'Orig_MaxTP':<10} {'Orig_RedLine':<12} {'Logic Applied':<45} {'Result':<8}\n")
        log_file.write("-" * 100 + "\n")
    
    for idx, trade in data.iterrows():
        trackback = trade['Trackback_Clean']
        max_tp = trade['MaxTP_Clean']
        red_line = trade['RedLine_Clean']
        
        if pd.isna(trackback) or pd.isna(max_tp) or pd.isna(red_line):
            continue
        
        # Initialize variables for logging
        logic_applied = ""
        
        # Simulation logic with detailed logging
        if red_line == "lose":
            # Original loser - check if wider stop saves it
            if trackback != -1.0 and trackback < atr_stop_level:
                # Could be saved, but what would the result be?
                if is_redline:
                    # For redline, if saved, check if original MaxTP reached our win_lose target (in original 1.0x ATR terms)
                    if max_tp >= original_win_lose_r:  # Compare in original ATR terms!
                        risk_ratio = target_r_current_atr  # Take the target in current ATR terms
                        logic_applied = f"Saved, MaxTP {max_tp:.1f}x >= {original_win_lose_r:.1f}x target, take {target_r_current_atr:.1f}R"
                    else:
                        risk_ratio = -1.0  # Saved but didn't reach win_lose target in ATR terms
                        logic_applied = f"Saved but MaxTP {max_tp:.1f}x < {original_win_lose_r:.1f}x target"
                else:
                    # Fixed R target
                    potential_max = max_tp / atr_stop_level if max_tp > 0 else 0
                    if potential_max >= r_target:
                        risk_ratio = r_target  # Hit our R target
                        logic_applied = f"Saved by wide stop, hit {r_target:.1f}R target"
                    else:
                        risk_ratio = -1.0  # Saved but didn't reach target
                        logic_applied = f"Saved but missed target: {potential_max:.2f} < {r_target:.1f}R"
            else:
                risk_ratio = -1.0  # Still a loser
                if trackback == -1.0:
                    logic_applied = "Original loser (-1), still loses"
                else:
                    logic_applied = f"Trackback {trackback:.1f} >= stop {atr_stop_level:.1f}, stopped"
                
        else:
            # Original winner (win_win or win_lose)
            if trackback >= atr_stop_level:
                risk_ratio = -1.0  # New stop would stop us out
                logic_applied = f"Stopped out: trackback {trackback:.1f} >= stop {atr_stop_level:.1f}"
            else:
                # Trade survives, calculate result based on target type
                if is_redline:
                    # Use enhanced redline strategy with configurable win_lose R
                    if red_line == "win_win":
                        # Originally got full profit to redline
                        risk_ratio = max_tp / atr_stop_level
                        logic_applied = f"Redline win_win: {max_tp:.1f}x/{atr_stop_level:.1f} = {risk_ratio:.2f}R"
                    elif red_line == "win_lose":
                        # For win_lose: Check if original MaxTP reached our win_lose target (in original 1.0x ATR terms)
                        if max_tp >= original_win_lose_r:  # Compare in original ATR terms!
                            risk_ratio = target_r_current_atr  # Take the target in current ATR terms
                            logic_applied = f"Redline win_lose HIT: MaxTP {max_tp:.1f}x >= {original_win_lose_r:.1f}x target, take {target_r_current_atr:.1f}R"
                        else:
                            risk_ratio = -1.0  # Didn't reach our win_lose target in original ATR terms
                            logic_applied = f"Redline win_lose MISS: MaxTP {max_tp:.1f}x < {original_win_lose_r:.1f}x target"
                else:
                    # Fixed R target strategy
                    potential_max = max_tp / atr_stop_level
                    
                    if potential_max >= r_target:
                        risk_ratio = r_target  # Hit our R target
                        logic_applied = f"Hit R target: {potential_max:.2f} >= {r_target:.1f}, take {r_target:.1f}R"
                    else:
                        # Didn't reach our R target - NO PROFIT TAKEN
                        risk_ratio = -1.0  # Either stopped out or didn't reach target
                        logic_applied = f"Missed R target: {potential_max:.2f} < {r_target:.1f}R, no profit"
        
        results.append(risk_ratio)
        
        # Store detailed log
        trade_log = {
            'Trade_Idx': idx,
            'ATR_Stop': atr_stop_level,
            'R_Target': target_display,
            'Original_Trackback': trackback,
            'Original_MaxTP': max_tp,
            'Original_RedLine': red_line,
            'Logic_Applied': logic_applied,
            'Final_RR': risk_ratio
        }
        trade_logs.append(trade_log)
        
        if log_file:
            log_file.write(f"{idx:<5} {trackback:<8} {max_tp:<10} {red_line:<12} {logic_applied:<45} {risk_ratio:+8.2f}\n")
    
    # Calculate summary metrics
    total_trades = len(results)
    winners = [rr for rr in results if rr > 0]
    losers = [rr for rr in results if rr <= 0]
    
    win_rate = len(winners) / total_trades * 100 if total_trades > 0 else 0
    avg_winner = np.mean(winners) if winners else 0
    avg_loser = abs(np.mean(losers)) if losers else 1
    expectancy = np.mean(results) if results else 0
    total_profit = sum(winners) if winners else 0
    profit_factor = total_profit / abs(sum(losers)) if losers else 0
    
    if log_file:
        log_file.write(f"\n📊 SUMMARY:\n")
        log_file.write(f"   • Total Trades: {total_trades}\n")
        log_file.write(f"   • Winners: {len(winners)} | Losers: {len(losers)}\n")
        log_file.write(f"   • Win Rate: {win_rate:.1f}%\n")
        log_file.write(f"   • Avg Winner: {avg_winner:.3f}R\n")
        log_file.write(f"   • Avg Loser: {avg_loser:.3f}R\n")
        log_file.write(f"   • Total Profit: {total_profit:.2f}R\n")
        log_file.write(f"   • Expectancy: {expectancy:+.3f}R per trade\n")
        log_file.write(f"   • Profit Factor: {profit_factor:.2f}\n")
        log_file.write("\n" + "="*100 + "\n")
    
    return {
        'ATR_Stop': atr_stop_level,
        'R_Target': target_display,
        'Win_Rate': win_rate,
        'Avg_Winner': avg_winner,
        'Avg_Loser': avg_loser,
        'Expectancy': expectancy,
        'Profit_Factor': profit_factor,
        'Total_Profit': total_profit,
        'Winners': len(winners),
        'Losers': len(losers),
        'Trade_Logs': trade_logs
    }

# Complete optimization ranges - Simplified approach for clean increments
atr_levels = np.arange(0.6, 3.1, 0.1).round(1).tolist()  # 0.6 to 3.0 in 0.1 increments

# Define clean target ranges for both redline and fixed strategies
clean_targets = np.arange(1.0, 5.1, 0.2).round(1).tolist()  # 1.0, 1.2, 1.4, ... 5.0

# Generate all combinations
all_combinations = []

for atr_level in atr_levels:
    # Add redline strategies with clean targets (in current ATR terms)
    for target_r in clean_targets:
        redline_strategy = f"redline_clean_{target_r:.1f}"
        all_combinations.append((atr_level, redline_strategy))
    
    # Add fixed R targets
    for fixed_r in clean_targets:
        all_combinations.append((atr_level, fixed_r))

total_combinations = len(all_combinations)

print(f"🚀 RUNNING COMPLETE OPTIMIZATION WITH FULL LOGGING")
print(f"📊 Testing {total_combinations} combinations")
print(f"🎯 Clean target increments: 1.0R, 1.2R, 1.4R, etc. (no decimals like 4.67R)")
print(f"   • Redline strategies: Targets in current ATR terms") 
print(f"   • Fixed R targets: Binary outcome (hit target or -1R)")
print(f"📁 This will generate detailed logs for ALL combinations...")

# Create timestamped filenames
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
detailed_log_filename = f"complete_atr_optimization_detailed_{timestamp}.txt"
summary_filename = f"complete_atr_optimization_summary_{timestamp}.txt"
results_csv_filename = f"complete_atr_optimization_results_{timestamp}.csv"

print(f"📝 Detailed log: {detailed_log_filename}")
print(f"📊 Summary: {summary_filename}")
print(f"📈 Results CSV: {results_csv_filename}")

# Run complete optimization with logging
optimization_results = []

# Open detailed log file
with open(detailed_log_filename, 'w', encoding='utf-8') as detailed_log:
    detailed_log.write("🔍 COMPLETE ATR + R TARGET OPTIMIZATION - DETAILED LOG\n")
    detailed_log.write("=" * 80 + "\n")
    detailed_log.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    detailed_log.write(f"Dataset: {len(valid_enhanced)} valid trades\n")
    detailed_log.write(f"ATR Range: {atr_levels[0]:.1f} to {atr_levels[-1]:.1f} (increments: 0.1)\n")
    detailed_log.write(f"Target Range: {clean_targets[0]:.1f} to {clean_targets[-1]:.1f} (increments: 0.2)\n")
    detailed_log.write(f"Total Combinations: {total_combinations}\n")
    detailed_log.write("=" * 80 + "\n")
    detailed_log.write("🎯 CLEAN REDLINE STRATEGIES:\n")
    detailed_log.write("   • win_win trades = full MaxTP (scaled for ATR stop)\n") 
    detailed_log.write("   • win_lose trades = clean R targets (1.0R, 1.2R, 1.4R, etc.)\n")
    detailed_log.write("   • Targets shown in current ATR terms (no messy decimals)\n")
    detailed_log.write("🎯 FIXED R TARGETS: Binary outcome (hit exact target = +R, miss target = -1R)\n")
    detailed_log.write("=" * 80 + "\n\n")
    
    # Run all combinations
    for combination_count, (atr_level, r_target) in enumerate(all_combinations, 1):
        
        # Show progress
        if combination_count % 100 == 0:
            print(f"📈 Progress: {combination_count}/{total_combinations} combinations completed ({combination_count/total_combinations*100:.1f}%)")
        
        # Run simulation with logging
        result = simulate_atr_and_r_target_with_logging(
            valid_enhanced, atr_level, r_target, detailed_log
        )
        
        optimization_results.append(result)

print(f"✅ Completed all {total_combinations} combinations!")

# Convert to DataFrame for analysis
opt_df = pd.DataFrame([{k: v for k, v in result.items() if k != 'Trade_Logs'} 
                       for result in optimization_results])

# Save results to CSV
opt_df.to_csv(results_csv_filename, index=False)

# Find top results
top_20_expectancy = opt_df.nlargest(20, 'Expectancy')
top_20_profit = opt_df.nlargest(20, 'Total_Profit')
top_20_winrate = opt_df.nlargest(20, 'Win_Rate')

# Create summary file
with open(summary_filename, 'w', encoding='utf-8') as summary:
    summary.write("🎯 COMPLETE ATR + R TARGET OPTIMIZATION - SUMMARY\n")
    summary.write("=" * 60 + "\n")
    summary.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    summary.write(f"Total combinations tested: {total_combinations}\n")
    summary.write(f"Dataset: {len(valid_enhanced)} valid trades\n")
    summary.write(f"Clean target increments: 1.0R, 1.2R, 1.4R, etc.\n\n")
    
    summary.write("🎯 TARGET EXPLANATIONS:\n")
    summary.write("   • Redline(X.XR): win_win=full TP, win_lose=X.XR (clean increments)\n")
    summary.write("   • Fixed R: Binary outcome (hit exact target or -1R loss)\n\n")
    
    # Best overall results
    best_expectancy = opt_df.loc[opt_df['Expectancy'].idxmax()]
    best_profit = opt_df.loc[opt_df['Total_Profit'].idxmax()]
    best_winrate = opt_df.loc[opt_df['Win_Rate'].idxmax()]
    
    summary.write("🏆 OPTIMAL RESULTS:\n")
    summary.write("-" * 40 + "\n")
    summary.write(f"🎯 Best Expectancy: {best_expectancy['ATR_Stop']:.1f}x ATR + {best_expectancy['R_Target']} target\n")
    summary.write(f"   Expectancy: {best_expectancy['Expectancy']:+.3f}R per trade\n")
    summary.write(f"   Win Rate: {best_expectancy['Win_Rate']:.1f}%\n")
    summary.write(f"   Total Profit: {best_expectancy['Total_Profit']:.2f}R\n\n")
    
    summary.write(f"💰 Highest Total Profit: {best_profit['ATR_Stop']:.1f}x ATR + {best_profit['R_Target']} target\n")
    summary.write(f"   Total Profit: {best_profit['Total_Profit']:.2f}R\n")
    summary.write(f"   Expectancy: {best_profit['Expectancy']:+.3f}R\n")
    summary.write(f"   Win Rate: {best_profit['Win_Rate']:.1f}%\n\n")
    
    summary.write(f"🎪 Highest Win Rate: {best_winrate['ATR_Stop']:.1f}x ATR + {best_winrate['R_Target']} target\n")
    summary.write(f"   Win Rate: {best_winrate['Win_Rate']:.1f}%\n")
    summary.write(f"   Expectancy: {best_winrate['Expectancy']:+.3f}R\n")
    summary.write(f"   Total Profit: {best_winrate['Total_Profit']:.2f}R\n\n")
    
    # Top 20 by expectancy
    summary.write("📊 TOP 20 COMBINATIONS BY EXPECTANCY:\n")
    summary.write("-" * 80 + "\n")
    summary.write(f"{'Rank':<4} {'ATR':<6} {'Target':<15} {'Expectancy':<10} {'Win Rate':<8} {'Total Profit':<12}\n")
    summary.write("-" * 80 + "\n")
    
    for i, (_, row) in enumerate(top_20_expectancy.iterrows(), 1):
        target_str = str(row['R_Target'])[:14]  # Truncate if too long
        summary.write(f"{i:<4} {row['ATR_Stop']:<6.1f} {target_str:<15} {row['Expectancy']:<10.3f} {row['Win_Rate']:<8.1f}% {row['Total_Profit']:<12.2f}\n")

print(f"\n🎊 COMPLETE OPTIMIZATION FINISHED!")
print("=" * 50)
print(f"📁 Files created:")
print(f"   • {detailed_log_filename} - Complete trade-by-trade logs")
print(f"   • {summary_filename} - Key results and top combinations")
print(f"   • {results_csv_filename} - All results in spreadsheet format")

print(f"\n🏆 BEST RESULTS:")
print(f"🎯 Optimal: {best_expectancy['ATR_Stop']:.1f}x ATR + {best_expectancy['R_Target']} target (Expectancy: {best_expectancy['Expectancy']:+.3f})")
print(f"💰 Max Profit: {best_profit['ATR_Stop']:.1f}x ATR + {best_profit['R_Target']} target (Profit: {best_profit['Total_Profit']:.2f}R)")

print(f"\n✨ Review the detailed log to verify all calculations are correct!")

🚀 RUNNING COMPLETE OPTIMIZATION WITH FULL LOGGING
📊 Testing 1050 combinations
🎯 Clean target increments: 1.0R, 1.2R, 1.4R, etc. (no decimals like 4.67R)
   • Redline strategies: Targets in current ATR terms
   • Fixed R targets: Binary outcome (hit target or -1R)
📁 This will generate detailed logs for ALL combinations...
📝 Detailed log: complete_atr_optimization_detailed_20250607_235814.txt
📊 Summary: complete_atr_optimization_summary_20250607_235814.txt
📈 Results CSV: complete_atr_optimization_results_20250607_235814.csv
📈 Progress: 100/1050 combinations completed (9.5%)
📈 Progress: 200/1050 combinations completed (19.0%)
📈 Progress: 300/1050 combinations completed (28.6%)
📈 Progress: 400/1050 combinations completed (38.1%)
📈 Progress: 500/1050 combinations completed (47.6%)
📈 Progress: 600/1050 combinations completed (57.1%)
📈 Progress: 700/1050 combinations completed (66.7%)
📈 Progress: 800/1050 combinations completed (76.2%)
📈 Progress: 900/1050 combinations completed (85.7%)
📈 Pro

In [59]:
# Complete ATR + R Target Optimization with FULL Logging for ALL Combinations
import pandas as pd
import numpy as np
import sys
from datetime import datetime

def simulate_atr_and_r_target_with_logging(data, atr_stop_level, r_target, log_file=None):
    """
    Simulate performance with detailed logging for verification
    r_target can be:
    - A number (e.g., 2.0) for fixed R target strategy
    - "redline_clean_X.X" (e.g., "redline_clean_2.4") for redline strategy with X.X R target (in current ATR terms)
    """
    results = []
    trade_logs = []
    
    # Parse target type
    if isinstance(r_target, str) and r_target.startswith("redline_clean_"):
        is_redline = True
        target_r_current_atr = float(r_target.split("_")[2])  # This is already in current ATR terms
        # Convert to original 1.0x ATR terms for comparison with MaxTP
        original_win_lose_r = target_r_current_atr * atr_stop_level
        target_display = f"Redline({target_r_current_atr:.1f}R)"
    else:
        is_redline = False
        original_win_lose_r = None
        target_r_current_atr = r_target  # Already in current ATR terms
        target_display = f"{r_target:.1f}R"
    
    if log_file:
        log_file.write(f"\n🔍 COMBINATION: {atr_stop_level:.1f}x ATR + {target_display} target\n")
        log_file.write("=" * 100 + "\n")
        log_file.write(f"{'Trade':<5} {'Orig_TB':<8} {'Orig_MaxTP':<10} {'Orig_RedLine':<12} {'Logic Applied':<45} {'Result':<8}\n")
        log_file.write("-" * 100 + "\n")
    
    for idx, trade in data.iterrows():
        trackback = trade['Trackback_Clean']
        max_tp = trade['MaxTP_Clean']
        red_line = trade['RedLine_Clean']
        
        if pd.isna(trackback) or pd.isna(max_tp) or pd.isna(red_line):
            continue
        
        # Initialize variables for logging
        logic_applied = ""
        
        # Simulation logic with detailed logging
        if red_line == "lose":
            # Original loser - check if wider stop saves it
            if trackback != -1.0 and trackback < atr_stop_level:
                # Could be saved, but what would the result be?
                if is_redline:
                    # For redline, if saved, what would the result be?
                    # We treat saved losers as potential win_lose trades
                    if max_tp >= original_win_lose_r:  # Compare in original ATR terms!
                        risk_ratio = target_r_current_atr  # Take the target in current ATR terms
                        logic_applied = f"Saved, MaxTP {max_tp:.1f}x >= {original_win_lose_r:.1f}x target, take {target_r_current_atr:.1f}R"
                    else:
                        risk_ratio = -1.0  # Saved but didn't reach win_lose target in ATR terms
                        logic_applied = f"Saved but MaxTP {max_tp:.1f}x < {original_win_lose_r:.1f}x target"
                else:
                    # Fixed R target
                    potential_max = max_tp / atr_stop_level if max_tp > 0 else 0
                    if potential_max >= r_target:
                        risk_ratio = r_target  # Hit our R target
                        logic_applied = f"Saved by wide stop, hit {r_target:.1f}R target"
                    else:
                        risk_ratio = -1.0  # Saved but didn't reach target
                        logic_applied = f"Saved but missed target: {potential_max:.2f} < {r_target:.1f}R"
            else:
                risk_ratio = -1.0  # Still a loser
                if trackback == -1.0:
                    logic_applied = "Original loser (-1), still loses"
                else:
                    logic_applied = f"Trackback {trackback:.1f} >= stop {atr_stop_level:.1f}, stopped"
                
        else:
            # Original winner (win_win or win_lose)
            if trackback >= atr_stop_level:
                risk_ratio = -1.0  # New stop would stop us out
                logic_applied = f"Stopped out: trackback {trackback:.1f} >= stop {atr_stop_level:.1f}"
            else:
                # Trade survives, calculate result based on target type
                if is_redline:
                    # Use enhanced redline strategy
                    if red_line == "win_win":
                        # win_win means they actually reached their ORIGINAL redline and exited
                        # They get their actual exit profit regardless of new target
                        risk_ratio = max_tp / atr_stop_level
                        logic_applied = f"Redline win_win: Got actual exit at {max_tp:.1f}x = {risk_ratio:.2f}R"
                    elif red_line == "win_lose":
                        # win_lose means they missed original redline, only got 1R
                        # Now check if they would reach our NEW target
                        if max_tp >= original_win_lose_r:  # Compare in original ATR terms!
                            risk_ratio = target_r_current_atr  # Take the target in current ATR terms
                            logic_applied = f"Redline win_lose HIT: MaxTP {max_tp:.1f}x >= {original_win_lose_r:.1f}x target, take {target_r_current_atr:.1f}R"
                        else:
                            risk_ratio = -1.0  # Didn't reach our win_lose target in original ATR terms
                            logic_applied = f"Redline win_lose MISS: MaxTP {max_tp:.1f}x < {original_win_lose_r:.1f}x target"
                else:
                    # Fixed R target strategy
                    potential_max = max_tp / atr_stop_level
                    
                    if potential_max >= r_target:
                        risk_ratio = r_target  # Hit our R target
                        logic_applied = f"Hit R target: {potential_max:.2f} >= {r_target:.1f}, take {r_target:.1f}R"
                    else:
                        # Didn't reach our R target - NO PROFIT TAKEN
                        risk_ratio = -1.0  # Either stopped out or didn't reach target
                        logic_applied = f"Missed R target: {potential_max:.2f} < {r_target:.1f}R, no profit"
        
        results.append(risk_ratio)
        
        # Store detailed log
        trade_log = {
            'Trade_Idx': idx,
            'ATR_Stop': atr_stop_level,
            'R_Target': target_display,
            'Original_Trackback': trackback,
            'Original_MaxTP': max_tp,
            'Original_RedLine': red_line,
            'Logic_Applied': logic_applied,
            'Final_RR': risk_ratio
        }
        trade_logs.append(trade_log)
        
        if log_file:
            log_file.write(f"{idx:<5} {trackback:<8} {max_tp:<10} {red_line:<12} {logic_applied:<45} {risk_ratio:+8.2f}\n")
    
    # Calculate summary metrics
    total_trades = len(results)
    winners = [rr for rr in results if rr > 0]
    losers = [rr for rr in results if rr <= 0]
    
    win_rate = len(winners) / total_trades * 100 if total_trades > 0 else 0
    avg_winner = np.mean(winners) if winners else 0
    avg_loser = abs(np.mean(losers)) if losers else 1
    expectancy = np.mean(results) if results else 0
    total_profit = sum(winners) if winners else 0
    profit_factor = total_profit / abs(sum(losers)) if losers else 0
    
    if log_file:
        log_file.write(f"\n📊 SUMMARY:\n")
        log_file.write(f"   • Total Trades: {total_trades}\n")
        log_file.write(f"   • Winners: {len(winners)} | Losers: {len(losers)}\n")
        log_file.write(f"   • Win Rate: {win_rate:.1f}%\n")
        log_file.write(f"   • Avg Winner: {avg_winner:.3f}R\n")
        log_file.write(f"   • Avg Loser: {avg_loser:.3f}R\n")
        log_file.write(f"   • Total Profit: {total_profit:.2f}R\n")
        log_file.write(f"   • Expectancy: {expectancy:+.3f}R per trade\n")
        log_file.write(f"   • Profit Factor: {profit_factor:.2f}\n")
        log_file.write("\n" + "="*100 + "\n")
    
    return {
        'ATR_Stop': atr_stop_level,
        'R_Target': target_display,
        'Win_Rate': win_rate,
        'Avg_Winner': avg_winner,
        'Avg_Loser': avg_loser,
        'Expectancy': expectancy,
        'Profit_Factor': profit_factor,
        'Total_Profit': total_profit,
        'Winners': len(winners),
        'Losers': len(losers),
        'Trade_Logs': trade_logs
    }

# Complete optimization ranges - Simplified approach for clean increments
atr_levels = np.arange(0.6, 3.1, 0.1).round(1).tolist()  # 0.6 to 3.0 in 0.1 increments

# Define clean target ranges for both redline and fixed strategies
clean_targets = np.arange(1.0, 5.1, 0.2).round(1).tolist()  # 1.0, 1.2, 1.4, ... 5.0

# Generate all combinations
all_combinations = []

for atr_level in atr_levels:
    # Add redline strategies with clean targets (in current ATR terms)
    for target_r in clean_targets:
        redline_strategy = f"redline_clean_{target_r:.1f}"
        all_combinations.append((atr_level, redline_strategy))
    
    # Add fixed R targets
    for fixed_r in clean_targets:
        all_combinations.append((atr_level, fixed_r))

total_combinations = len(all_combinations)

print(f"🚀 RUNNING COMPLETE OPTIMIZATION WITH FULL LOGGING")
print(f"📊 Testing {total_combinations} combinations")
print(f"🎯 Clean target increments: 1.0R, 1.2R, 1.4R, etc. (no decimals like 4.67R)")
print(f"   • Redline strategies: win_win=actual exits, win_lose=test new targets") 
print(f"   • Fixed R targets: Binary outcome (hit target or -1R)")
print(f"📁 This will generate detailed logs for ALL combinations...")

# Create timestamped filenames
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
detailed_log_filename = f"complete_atr_optimization_detailed_{timestamp}.txt"
summary_filename = f"complete_atr_optimization_summary_{timestamp}.txt"
results_csv_filename = f"complete_atr_optimization_results_{timestamp}.csv"

print(f"📝 Detailed log: {detailed_log_filename}")
print(f"📊 Summary: {summary_filename}")
print(f"📈 Results CSV: {results_csv_filename}")

# Run complete optimization with logging
optimization_results = []

# Open detailed log file
with open(detailed_log_filename, 'w', encoding='utf-8') as detailed_log:
    detailed_log.write("🔍 COMPLETE ATR + R TARGET OPTIMIZATION - DETAILED LOG\n")
    detailed_log.write("=" * 80 + "\n")
    detailed_log.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    detailed_log.write(f"Dataset: {len(valid_enhanced)} valid trades\n")
    detailed_log.write(f"ATR Range: {atr_levels[0]:.1f} to {atr_levels[-1]:.1f} (increments: 0.1)\n")
    detailed_log.write(f"Target Range: {clean_targets[0]:.1f} to {clean_targets[-1]:.1f} (increments: 0.2)\n")
    detailed_log.write(f"Total Combinations: {total_combinations}\n")
    detailed_log.write("=" * 80 + "\n")
    detailed_log.write("🎯 CORRECT REDLINE LOGIC:\n")
    detailed_log.write("   • win_win trades = actual exits at original redline (get full profit)\n") 
    detailed_log.write("   • win_lose trades = missed original redline (test against new target)\n")
    detailed_log.write("   • Targets shown in current ATR terms (clean increments)\n")
    detailed_log.write("🎯 FIXED R TARGETS: Binary outcome (hit exact target = +R, miss target = -1R)\n")
    detailed_log.write("=" * 80 + "\n\n")
    
    # Run all combinations
    for combination_count, (atr_level, r_target) in enumerate(all_combinations, 1):
        
        # Show progress
        if combination_count % 100 == 0:
            print(f"📈 Progress: {combination_count}/{total_combinations} combinations completed ({combination_count/total_combinations*100:.1f}%)")
        
        # Run simulation with logging
        result = simulate_atr_and_r_target_with_logging(
            valid_enhanced, atr_level, r_target, detailed_log
        )
        
        optimization_results.append(result)

print(f"✅ Completed all {total_combinations} combinations!")

# Convert to DataFrame for analysis
opt_df = pd.DataFrame([{k: v for k, v in result.items() if k != 'Trade_Logs'} 
                       for result in optimization_results])

# Save results to CSV
opt_df.to_csv(results_csv_filename, index=False)

# Find top results
top_20_expectancy = opt_df.nlargest(20, 'Expectancy')
top_20_profit = opt_df.nlargest(20, 'Total_Profit')
top_20_winrate = opt_df.nlargest(20, 'Win_Rate')

# Create summary file
with open(summary_filename, 'w', encoding='utf-8') as summary:
    summary.write("🎯 COMPLETE ATR + R TARGET OPTIMIZATION - SUMMARY\n")
    summary.write("=" * 60 + "\n")
    summary.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    summary.write(f"Total combinations tested: {total_combinations}\n")
    summary.write(f"Dataset: {len(valid_enhanced)} valid trades\n")
    summary.write(f"Clean target increments: 1.0R, 1.2R, 1.4R, etc.\n\n")
    
    summary.write("🎯 TARGET EXPLANATIONS:\n")
    summary.write("   • Redline(X.XR): win_win=actual exits, win_lose=test new target\n")
    summary.write("   • Fixed R: Binary outcome (hit exact target or -1R loss)\n\n")
    
    # Best overall results
    best_expectancy = opt_df.loc[opt_df['Expectancy'].idxmax()]
    best_profit = opt_df.loc[opt_df['Total_Profit'].idxmax()]
    best_winrate = opt_df.loc[opt_df['Win_Rate'].idxmax()]
    
    summary.write("🏆 OPTIMAL RESULTS:\n")
    summary.write("-" * 40 + "\n")
    summary.write(f"🎯 Best Expectancy: {best_expectancy['ATR_Stop']:.1f}x ATR + {best_expectancy['R_Target']} target\n")
    summary.write(f"   Expectancy: {best_expectancy['Expectancy']:+.3f}R per trade\n")
    summary.write(f"   Win Rate: {best_expectancy['Win_Rate']:.1f}%\n")
    summary.write(f"   Total Profit: {best_expectancy['Total_Profit']:.2f}R\n\n")
    
    summary.write(f"💰 Highest Total Profit: {best_profit['ATR_Stop']:.1f}x ATR + {best_profit['R_Target']} target\n")
    summary.write(f"   Total Profit: {best_profit['Total_Profit']:.2f}R\n")
    summary.write(f"   Expectancy: {best_profit['Expectancy']:+.3f}R\n")
    summary.write(f"   Win Rate: {best_profit['Win_Rate']:.1f}%\n\n")
    
    summary.write(f"🎪 Highest Win Rate: {best_winrate['ATR_Stop']:.1f}x ATR + {best_winrate['R_Target']} target\n")
    summary.write(f"   Win Rate: {best_winrate['Win_Rate']:.1f}%\n")
    summary.write(f"   Expectancy: {best_winrate['Expectancy']:+.3f}R\n")
    summary.write(f"   Total Profit: {best_winrate['Total_Profit']:.2f}R\n\n")
    
    # Top 20 by expectancy
    summary.write("📊 TOP 20 COMBINATIONS BY EXPECTANCY:\n")
    summary.write("-" * 80 + "\n")
    summary.write(f"{'Rank':<4} {'ATR':<6} {'Target':<15} {'Expectancy':<10} {'Win Rate':<8} {'Total Profit':<12}\n")
    summary.write("-" * 80 + "\n")
    
    for i, (_, row) in enumerate(top_20_expectancy.iterrows(), 1):
        target_str = str(row['R_Target'])[:14]  # Truncate if too long
        summary.write(f"{i:<4} {row['ATR_Stop']:<6.1f} {target_str:<15} {row['Expectancy']:<10.3f} {row['Win_Rate']:<8.1f}% {row['Total_Profit']:<12.2f}\n")

print(f"\n🎊 COMPLETE OPTIMIZATION FINISHED!")
print("=" * 50)
print(f"📁 Files created:")
print(f"   • {detailed_log_filename} - Complete trade-by-trade logs")
print(f"   • {summary_filename} - Key results and top combinations")
print(f"   • {results_csv_filename} - All results in spreadsheet format")

print(f"\n🏆 BEST RESULTS:")
print(f"🎯 Optimal: {best_expectancy['ATR_Stop']:.1f}x ATR + {best_expectancy['R_Target']} target (Expectancy: {best_expectancy['Expectancy']:+.3f})")
print(f"💰 Max Profit: {best_profit['ATR_Stop']:.1f}x ATR + {best_profit['R_Target']} target (Profit: {best_profit['Total_Profit']:.2f}R)")

print(f"\n✨ Review the detailed log to verify all calculations are correct!")

🚀 RUNNING COMPLETE OPTIMIZATION WITH FULL LOGGING
📊 Testing 1050 combinations
🎯 Clean target increments: 1.0R, 1.2R, 1.4R, etc. (no decimals like 4.67R)
   • Redline strategies: win_win=actual exits, win_lose=test new targets
   • Fixed R targets: Binary outcome (hit target or -1R)
📁 This will generate detailed logs for ALL combinations...
📝 Detailed log: complete_atr_optimization_detailed_20250608_000131.txt
📊 Summary: complete_atr_optimization_summary_20250608_000131.txt
📈 Results CSV: complete_atr_optimization_results_20250608_000131.csv
📈 Progress: 100/1050 combinations completed (9.5%)
📈 Progress: 200/1050 combinations completed (19.0%)
📈 Progress: 300/1050 combinations completed (28.6%)
📈 Progress: 400/1050 combinations completed (38.1%)
📈 Progress: 500/1050 combinations completed (47.6%)
📈 Progress: 600/1050 combinations completed (57.1%)
📈 Progress: 700/1050 combinations completed (66.7%)
📈 Progress: 800/1050 combinations completed (76.2%)
📈 Progress: 900/1050 combinations comp